## Angry Birds - Final Assignment
Student number 1504668

In this task we will be attempting to recreate the popular mobile game "angry birds" by launching a projectile into an object and, using relavent physics calculating whether the applied torque of the collision is enough to cause the object to topple.

To do this we will be using these equations of motion $$ x=x_0 + v_0tcos(\theta)$$ for the Horizontal displacement $$ y=y_0+v_0tsin(\theta) - \frac {gt^2}{2}$$ for the vertical displacement

Hence the momentum of the projectile can be expressed as 
$$ p_x=mv_ocos(\theta)$$ Horizontally and $$ p_y = mv_0sin(\theta) - mgt$$ Vertically

On collision the force exerted can be shown as $$ F_{applied} = \frac{\Delta p}{\Delta t} $$

The applied Torque is then $$\tau_{applied} = F_{applied} \times d_a $$
where $d_a$ is the vector from the point of collision to the point of rotation

if the applied Torque is greater than the restoring torque of the block $$ \tau_{restoring} = -mg\frac{w}{2} $$
where w is the width of the block

Then the block will topple.

I will use the above outlined theory in order to calculate whether the player of the the game wins. We will assume that 100% of the momentum is transferred into the block in a timestep of 0.01 seconds.
We also assume that the position of the block is fixed and will only rotate in one direction

In [1]:
import numpy as np #import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, textures #import visual python

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
Game=True                                                                            #Sets initial requirement for game to run


In [3]:
while Game == True:
    scene = canvas(width=640, height=480, center=vector(8,0,0), range=8)                    #Setting Scene
    Ground = box(pos=vector(8,-0.125,0), length=16, height=0.15, width=5,texture=textures.metal) #Changed the ground to a box to explore
                                                                                            # Textures

    ############ Target parameters ##############
    targetpos = (random()*10)+5 #Assinging a random number between 5-15 for the x-position of the box
    target= box(pos=vector(targetpos,1,0),height = 2, width = 0.5, depth = 0.5, color = color.white) #Making the target
    targetmass = 100                                                                        #Target Mass
    ########### Projectile parameters ###########                                                                        
    m=0.1                                                                                   #Projectile/Bird mass
    r=0.05                                                                                  #Projectile/Bird Radius
    ########## Global parameters ################
    dT=0.001                                                                               #Timestep for animation
    dt=0.01                                                                                #Time for the force to act in collision
    g=9.81                                                                                 #Acceleration due to  Gravity
    hit_tolerance = r+0.25                                                                 #We assigned a hit tolerance a single variable
                                                                                           #Designed to calculate whether the bird hit the 
                                                                                           #Object, (surface of bird+ half width of object)
    
    
    win = False                                                                            # Set the initial condition to begin the game
    while win == False:
        x0=0                                                                               #Setting the initial starting x position
        y0=0                                                                               #Setting the initial starting y position
        y=y0                                                                               #Resetting the y postion after the loop has ran
        x=x0                                                                               #Resetting the x position after the loop has ran
        t = 0                                                                              #t=0 start of animation.
        projectile = sphere(pos = vector(x0,y0,0), radius = 0.3, make_trail=True, trail_type="points",interval = 1.0) #Form the projectile
                                                                                          #in vPython,
            
        ############ Asking the user for their inputs and whether they're suitable #####################
        ################## Angle ###############################
        iangle = float(input("Input the initial angle in degrees: "))                    
        while (iangle<0) or (iangle >90):
            print('Please Input a launch angle between 0 and 90')
            iangle = float(input("Input the initial angle in degrees: "))
        launchA = np.radians(iangle)                                                        #Convert input to radians
        
        ################# Velocity ############################
        launchV = float(input("Input the initial speed in metres/second: "))
        if launchV < 0:
            print("Please input a launch velocity greater than 0")
            launchV = float(input("Input the initial speed in metres/second: "))
       
    
       ############## Scene camera change ####################
        scene.camera.follow(projectile)                                                     #Makes the "action camera" follow the projectile
        scene.camera.axis = vector (-8,-2,-4)                                                # Adjusting the viewing axis 
        
        ################### labels ###########################
        L = label(pos=vector(3,-5,0))                                                       #Initial label position to give x,y,t updates
        Launch_label = label(pos=vector(4,-2,0))                                            # Initial Conditions label
        Launch_label.text = ("Launch Angle = {0:0.1f} Degrees Launch Velocity = {1:0.1f}".format(iangle,launchV))
        
        
        
        while (y >= y0) & (x<target.pos.x) & (y<50) & (x>-1) :                             #Setting conditions for animation to run
            x = x0 + launchV*t*np.cos(launchA)                                             #Calculating Horizontal position wrt Time
            y = y0 + launchV*t*np.sin(launchA) - (0.5*g*t**2)                              #Calculating Vertical Position wrt Time
            projectile.pos= vector (x,y,0)                                                 #Updating the projectile with the above calculations
            t=t+dT                                                                         #Increasing the t by the timestep dT
            L.text="x={0:0.3f}m y={1:0.3f}m t={2:0.3f}s".format(x,y,t)                     #Label giving realtime x,y pos wrt Time
            rate(60)                                                                       #Rate for animation to run
            
            
           
            
        #################### IF HIT ########################
            if (x>=target.pos.x-hit_tolerance) & (y<=2):  
        ############### Physics Calculation ################
                restoringT = abs(-targetmass*g*0.25)                                      #calculating the resotring torque                        
                xmomentum = m*launchV*np.cos(launchA)                                     #Calculating the resulting x momentum
                ymomentum = m*launchV*np.sin(launchA) - (m*g*t)                           #Calculating the resutling y momentum
                Tmomentum = mag(vector(xmomentum,ymomentum,0))                            #Total momementum is magnitude of the resultant vector
                da = vector (x+0.05,y+0.05,0)-vector(targetpos+0.25,0,0)                  #Vector bewtween point of collision and 
                                                                                          #Rotation point
                appliedT = ((vector(xmomentum,ymomentum,0))/dt).cross(da)                 #Calcualting the Applied Torque
                MagT = mag(appliedT)
                                                      
                
                
                
                
                
                if restoringT< abs(MagT):                                                 #Checking if the applied torque is greater than restoring
        ################### Attempt at rotation #############
        ## This is my attempt at rotation which works, however due to the replay feature does not occur until after 
        ## The user input is no. I do not know how to fix this. ## 
                    target.rotate(angle=(-pi/2),axis=vector(0,0,1), origin = vector((targetpos+0.25),0,0)) #Rotate through -90 degrees
                    target.color = color.green                                            #Makes the target object colour green on success
                    Win_label = label(pos=vector(8,5,0))
                    Win_label.text = "You Win!"                                           #Changes win label text to YOU WIN
                    
        ################# Printing results from above ############
                    print("Congratulations! You Toppled the block, Well done")            
                    print("Your impact height was {0:0.3f}m".format(y))                    #Impact height
                    print("The magnitude of momentum on impact was {0:0.3f}N".format(Tmomentum)) #Magnitude of momemntum
                    print("Resulting in an applied torque of {0:0.3f}Nm, Overcoming the restoring Torque of {1:0.2f}Nm".format(MagT,restoringT)) 
                    ## Resultant torque and restoring torque ##
                    answer = input("Play again (y/n)")                                   # Asking the user to play again
                    if answer == 'n':                                                    # No
                        print("Thanks for playing")
                        Game=False   
                        win = True                                                       #Fulfills loop req
                        break                                                            #breaks out of loop
                        
                    else:                                                                #Yes or otherwise
                        game=True
                        win = True                                                       #Resets game
                        break
                        
                    
            
        
        
                else:                                                                    #Not enough applied torque
                    print("You were unsucessful in toppling the block, please try again")
                    Win_label.visible = True
                    Win_label.text="Try Again!"                                          #Changes winlabel text
                    print("Your impact height was {0:0.3f}m".format(y))                  #Impact height
                    print("The magnitude of momentum on impact was {0:0.3f}N".format(Tmomentum)) #Magnitude of momemntum
                    print("Resulting in an applied torque of {0:0.3f}Nm, Not enough to overcome the restoring Torque of {1:0.2f}Nm".format(MagT,restoringT))
                    win = False                                                         #Requirement no fulfilled
                    scene.camera.axis = vector (0,0,-8)                                 #Attempt to reset camera, zoom our required
                    break
                   
                
            
    ##################### Missing the target ###################
        else:                                                          
            print("You Missed the Target, Try again!")
            Win_label = label(pos=vector(8,5,0))                                     #Adds the winlabel
            Win_label.text="Try Again!"                                                 #Changes text to Try again
            win = False                                                                 #Requirement not fulfilled
            scene.camera.axis = vector(0,0,-8)                                          #Resets the axis, zoom required
            
            
            
            
    
        
                        
            
        
    

        




<IPython.core.display.Javascript object>

Input the initial angle in degrees: 15
Input the initial speed in metres/second: 25
Congratulations! You Toppled the block, Well done
Your impact height was 1.177m
The magnitude of momentum on impact was 2.453N
Resulting in an applied torque of 317.475Nm, Overcoming the restoring Torque of 245.25Nm
Play again (y/n)n
Thanks!


My code follows a structure such that the game initialises > forms a canvas > sets the variables > begins animation - the while loop, to run the script while the projectile is in the air > checks if hit - making use of an if statement, such that the position of the projectile comes sufficently close to the block > if hit whether enough torque is applied - via another if loop applying the physics outlined in the introduction> if successful says you won > asks to play again > relaunches the canvas via resetting the over arching while loop for the game to run.

All other outcomes are considered losing and will simply replay the animation stage with new input variables

## Review of using Python to Model the Projectile Physics

Overall I feel VPython is more than capable of modelling the physics that occurs in a collision between two, or more objects. However is would be skeptical to trust the numerical outcomes, particularly for our attempt as we used a number of assumptions to simplify the process such as assuming 100% of the momementum is transferred into the object, which in reality rarely occurs, and the object intuitively should bounce off somewhat. Our chosen timestep can incur errors particularly at high velocities - although this can be fixed by adjusting the timestep, it is favourable to have an infintesimally small timestep in order to correctly identify the point of collision.

In terms of the "game" we used an object with much larger radius than that of the object used in our calculations, which reduces clarity for the player and as a result of this we assume that the masses are entirely uniform, and that all the mass of both objects is concentrated in their centres. A way of improving this is by calculating their moment of inertia. Building on this idea it could be possible to work out the angular velocity $\alpha = \frac{d^2 \theta }{dt^2}$ and hence have the block rotate proportional to the force applied, rather than an arbitraty rotation through 90 Degrees, this is related to the total torque $\tau_{applied} - \tau_{restoring}$ by $\tau = I \alpha $ where $I$ is the moment of inertia

To impliment a more accurate model in Python I believe that we should ask the user if they would like air resistance, therefore introducing extra decelleration in both directions, and reducing the acceleration due to gravity. This would aid to make the games more realistic, however we would have to assume that the resistive force was constant such that the equations of motion did not break.Furthermore if we could compute the angle of interaction between the object and the projectile at the point of collision we could more accurately calculate the components of momentum, and hence the force exerted on the block which can then be used to calculate acceleration as outlined in the second paragraph. Furthermore developing on the idea that it is rare for all the momentum to be transferred into the block, the block could be made of different materials, each with it's own coefficient of restitution, and hence would transfer variable amounts of force to the block, this can be implimented fairly easily as was done in session 8.

As an added extra, it would be possible to "replay" the collision with a much lower rate, and perhaps timestep allowing for a zoomed in scene to be created, allowing for a more precise model of the collision and the subsequent energy transfers. This can be done by adding another looping structure with the same initial conditions as the ones input by the user.

